In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.pardir, 'gotmtool'))
from gotmtool import *

In [ ]:
gotmtool_config = os.path.join(os.pardir, 'gotmtool', '.gotm_env.yaml')
gotm_env = config_load(gotmtool_config)

### Load the data

In [ ]:
casename = 'JunS'
inputdir = os.path.join(gotm_env['gotmdir_data'], 'SOFS', 'LES')
if casename == 'AprS':
    starttime = '2010-04-29 00:00:00'
    stoptime   = '2010-05-01 04:30:00'
    inputfile = 'les-april_2010.with_stokes.nc'
    levels_T  = np.linspace(11.1, 12.6, 31)
elif casename == "AprN":
    starttime = "2010-04-29 00:00:00"
    stoptime  = "2010-05-01 02:00:00"
    inputfile="les-april_2010.pp.no_stokes.nc"
    levels_T  = np.linspace(11.1, 12.6, 31)
elif casename == "JunS":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 23:45:00"
    inputfile = "les-june_2010.pp.with_stokes.nc"
    levels_T  = np.linspace(11.1, 11.9, 41) 
elif casename == "JunN":
    starttime = "2010-06-07 06:45:00"
    stoptime  = "2010-06-08 21:15:00"
    inputfile = "les-june_2010.pp.no_stokes.nc"
    levels_T  = np.linspace(11.1, 11.9, 41) 
else:
    raise ValueError('Case {:s} not supported. Stop.'.format(casename))

In [ ]:
# LES data
ds = xr.load_dataset(os.path.join(inputdir, inputfile))
dttime = pd.to_datetime(starttime)+pd.to_timedelta(ds.time, unit='s')
zu_m = -np.abs(ds.zu.values)*0.01
zu = xr.DataArray(zu_m,
                  dims=('zu'),
                  coords={'zu': zu_m},
                  attrs={'long_name': 'depth', 'units': 'm'})
zw_m = -np.abs(ds.zw.values)*0.01
zw = xr.DataArray(zw_m,
                  dims=('zw'),
                  coords={'zw': zw_m},
                  attrs={'long_name': 'depth', 'units': 'm'})
ds_les = ds.set_coords(('zu','zw'))
ds_les = ds_les.assign_coords(time=dttime, zu=zu, zw=zw)

In [ ]:
# GOTM data
turbmethods = ['KPPLT-LF17', 'StokesMOST', 'StokesMOST-r2']
abc = 'abcdefg'
labels = ['(a) LES ',]
gotmdir = gotm_env['gotmdir_run']
ds_gotm = []
for i, turbmethod in enumerate(turbmethods):
    sim = Simulation(path=os.path.join(gotmdir, 'SOFS-{:s}'.format(casename), turbmethod), dataname='gotm_out.nc')
    ds_gotm.append(sim.load_data())
    labels.append('({:s}) {:s}'.format(abc[i+1], turbmethod))

# load some constants
cfg = config_load(sim.config)
gotm_rho_0 = cfg['equation_of_state']['rho0']
gotm_cp = cfg['equation_of_state']['linear']['cp']

### Surface forcing 

In [ ]:
def multicolor_ylabel(ax,list_of_strings,list_of_colors,anchorpad=0,**kw):
    from matplotlib.offsetbox import AnchoredOffsetbox, TextArea, HPacker, VPacker

    # y-axis label
    boxes = [TextArea(text, textprops=dict(color=color, ha='left',va='bottom',rotation=90,**kw)) 
                 for text,color in zip(list_of_strings[::-1],list_of_colors[::-1]) ]
    ybox = VPacker(children=boxes,align="center", pad=0, sep=5)
    anchored_ybox = AnchoredOffsetbox(loc=3, child=ybox, pad=anchorpad, frameon=False, bbox_to_anchor=(-0.15, 0.15), 
                                      bbox_transform=ax.transAxes, borderpad=0.)
    ax.add_artist(anchored_ybox)

In [ ]:
ustar  = ds_les.data_vars['ustar'] * 0.01*20
laturb = ds_les.data_vars['La_t']
bflux  = -ds_les.data_vars['wbsfc']

fig = plt.figure(figsize=(6,3))
axis = plt.gca()
par1 = axis.twinx()
axis.axhline(0.3, color='navy', linewidth=0.75)
par1.axhline(0, color='darkred', linewidth=0.75)

ustar.plot(ax=axis, color='black', linewidth=1.5)
laturb.plot(ax=axis, color='steelblue', linewidth=1.5)
axis.set_xlabel('')
axis.set_ylabel('')
multicolor_ylabel(axis, ('$20 \\times u_*$ [m s$^{-1}$];','$\mathrm{La}_t$'),
                  ('k','steelblue'), size=11)

bflux.plot(ax=par1, color='lightcoral', linewidth=1.5)
par1.set_ylabel('$Q_0$ [W m$^{-2}$]', fontsize=11)
par1.yaxis.label.set_color('lightcoral')

axis.set_zorder(par1.get_zorder()+1)
axis.patch.set_visible(False)
axis.set_xlim([dttime[0], dttime[-1]])

plt.tight_layout()

### Profiles

In [ ]:
def plot_var_pfl(das, labels, **kwargs):
    ndat = len(das)
    fig, axarr = plt.subplots(ndat, 1, sharex='col')
    fig.set_size_inches([8,1+2*ndat])

    for i in np.arange(ndat):
        ax = axarr[i]
        das[i].plot(ax=ax, **kwargs)
        ax.text(0.03, 0.05, labels[i], transform=ax.transAxes, color='black', fontsize=11, va='bottom')
        ax.set_title('')
        ax.set_xlabel('')
        ax.set_ylabel('depth [m]')
        ax.set_ylim([-250,0])

    plt.tight_layout()
    # return fig

In [ ]:
levels = np.linspace(-0.2,0.2,41)

# LES
da0 = ds_les.data_vars['UVEL'].swap_dims({'nlev': 'zu'}).assign_coords({'zu': zu}).transpose('zu', 'time') * 0.01 # cm/s -> m/s
da0.attrs['long_name'] = '$u$'
da0.attrs['units'] = 'm/s'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = ds.data_vars['u']
    da.attrs['long_name'] = '$u$'
    da.attrs['units'] = 'm/s'
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
levels = np.linspace(-0.2,0.2,41)

# LES
da0 = ds_les.data_vars['VVEL'].swap_dims({'nlev': 'zu'}).assign_coords({'zu': zu}).transpose('zu', 'time') * 0.01 # cm/s -> m/s
da0.attrs['long_name'] = '$v$'
da0.attrs['units'] = 'm/s'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = ds.data_vars['v']
    da.attrs['long_name'] = '$v$'
    da.attrs['units'] = 'm/s'
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
levels = levels_T

# LES
da0 = ds_les.data_vars['TEMP'].swap_dims({'nlev': 'zu'}).assign_coords({'zu': zu}).transpose('zu', 'time')
da0.attrs['long_name'] = '$T$'
da0.attrs['units'] = '$^\circ$C'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = ds.data_vars['temp']
    da.attrs['long_name'] = '$T$'
    da.attrs['units'] = '$^\circ$C'
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
def compute_wt(ds_gotm):
    # temperature profiles (z)
    temp = ds_gotm.data_vars['temp'].data.squeeze()
    # turbulent diffusivity of heat (zi)
    nuh  = ds_gotm.data_vars['nuh'].data.squeeze()
    # nonlocal temperature flux (zi)
    gamh = ds_gotm.data_vars['gamh'].data.squeeze()
    # layer thickness (z)
    h    = ds_gotm.data_vars['h'].data.squeeze()
    # surface temperature flux
    wt0  = -ds_gotm.data_vars['heat'].data.squeeze()/gotm_cp/gotm_rho_0
    # vertical temperature flux
    wt = np.zeros(temp.shape)
    wt[-1,:] = wt0
    wt[0:-1,:] = -2.0*nuh[1:-1,:]*(temp[1:,:]-temp[0:-1,:])/(h[1:,:]+h[0:-1,:]) + gamh[1:-1,:]

    da = xr.DataArray(wt,
                       dims=('zi', 'time'),
                       coords={'time': ds_gotm.time, 'zi': ds_gotm.zi[1:]},
                       attrs={'long_name': '$\overline{w^\prime T^\prime}$', 'units': '$^\circ$C m/s'})
    return da

def compute_wu(ds_gotm):
    # velocity profiles (z)
    u = ds_gotm.data_vars['u'].data.squeeze()
    # turbulent diffusivity of momentum (zi)
    num  = ds_gotm.data_vars['num'].data.squeeze()
    # nonlocal momentum flux (zi)
    gamu = ds_gotm.data_vars['gamu'].data.squeeze()
    # layer thickness (z)
    h    = ds_gotm.data_vars['h'].data.squeeze()
    # surface momentum flux
    wu0  = -ds_gotm.data_vars['tx'].data.squeeze()
    # vertical momentum flux
    wu = np.zeros(u.shape)
    wu[-1,:] = wu0
    wu[0:-1,:] = -2.0*num[1:-1,:]*(u[1:,:]-u[0:-1,:])/(h[1:,:]+h[0:-1,:]) + gamu[1:-1,:]

    da = xr.DataArray(wu,
                       dims=('zi', 'time'),
                       coords={'time': ds_gotm.time, 'zi': ds_gotm.zi[1:]},
                       attrs={'long_name': '$\overline{w^\prime u^\prime}$', 'units': 'm$^2$/s$^2$'})
    return da

def compute_wv(ds_gotm):
    # velocity profiles (z)
    v = ds_gotm.data_vars['v'].data.squeeze()
    # turbulent diffusivity of momentum (zi)
    num  = ds_gotm.data_vars['num'].data.squeeze()
    # nonlocal momentum flux (zi)
    gamv = ds_gotm.data_vars['gamv'].data.squeeze()
    # layer thickness (z)
    h    = ds_gotm.data_vars['h'].data.squeeze()
    # surface momentum flux
    wv0  = -ds_gotm.data_vars['ty'].data.squeeze()
    # vertical momentum flux
    wv = np.zeros(v.shape)
    wv[-1,:] = wv0
    wv[0:-1,:] = -2.0*num[1:-1,:]*(v[1:,:]-v[0:-1,:])/(h[1:,:]+h[0:-1,:]) + gamv[1:-1,:]

    da = xr.DataArray(wv,
                       dims=('zi', 'time'),
                       coords={'time': ds_gotm.time, 'zi': ds_gotm.zi[1:]},
                       attrs={'long_name': '$\overline{w^\prime v^\prime}$', 'units': 'm$^2$/s$^2$'})
    return da

In [ ]:
levels = np.linspace(-6e-4,6e-4,61)

# LES
da0 = ds_les.data_vars['VTUF'].swap_dims({'nlev': 'zw'}).assign_coords({'zw': zw}).transpose('zw', 'time') * 1e-4 # cm^2/s^2 -> m^2/s^2
da0.attrs['long_name'] = '$\overline{w^\prime u^\prime}$'
da0.attrs['units'] = 'm$^2$/s$^2$'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = compute_wu(ds)
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
levels = np.linspace(-6e-4,6e-4,61)

# LES
da0 = ds_les.data_vars['VTVF'].swap_dims({'nlev': 'zw'}).assign_coords({'zw': zw}).transpose('zw', 'time') * 1e-4 # cm^2/s^2 -> m^2/s^2
da0.attrs['long_name'] = '$\overline{w^\prime v^\prime}$'
da0.attrs['units'] = 'm$^2$/s$^2$'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = compute_wv(ds)
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
levels = np.linspace(-1.2e-4,1.2e-4,61)

# LES
da0 = ds_les.data_vars['VTTF'].swap_dims({'nlev': 'zw'}).assign_coords({'zw': zw}).transpose('zw', 'time') * 0.01 # degC cm/s -> degC m/s
da0.attrs['long_name'] = '$\overline{w^\prime T^\prime}$'
da0.attrs['units'] = '$^\circ$C m/s'
das = [da0]

# GOTM
for i, ds in enumerate(ds_gotm):
    da = compute_wt(ds)
    das.append(da)

plot_var_pfl(das, labels, levels=levels)

In [ ]:
ds_gotm[-1].data_vars['StokesXi'].plot()